In [14]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..'))
import paths
from data_preparation.ScanNet.db_creation_scanNet_utils import load_as_pickle,save_as_pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.cluster import SpectralClustering, AffinityPropagation
from community import community_louvain  # Louvain
from leidenalg import find_partition, ModularityVertexPartition  # Leiden
import igraph as ig
from scipy.sparse.csgraph import connected_components
from sklearn.metrics import silhouette_score
import data_preparation.ScanNet.cath_utils as cath_utils

AttributeError: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)

In [4]:
dataset_date = "8_9"
seq_id = "0.95"
ASA_THRESHOLD_VALUE = 0.2
with_scanNet = False
with_scanNet_addition = '_with_scanNet_' if with_scanNet else ''
matHomologous_path = os.path.join(paths.cath_intermediate_files_path, f'matHomologous_{dataset_date}{with_scanNet_addition}.pkl')
matHomologous = load_as_pickle(matHomologous_path)
print(matHomologous.shape)
graphHomologous = csr_matrix(matHomologous) 
clusters_folder = os.path.join(paths.cath_intermediate_files_path, f'clusters{with_scanNet_addition}')
os.makedirs(clusters_folder, exist_ok=True)

(468, 468)


In [3]:
def reorder_labels_by_cluster_size(labels):
    """Reorder labels such that label 0 is for the biggest cluster, label 1 for the second biggest, and so on."""
    unique, counts = np.unique(labels, return_counts=True)
    sorted_indices = np.argsort(-counts)  # Sort in descending order
    new_labels = np.zeros_like(labels)
    for new_label, old_label in enumerate(unique[sorted_indices]):
        new_labels[labels == old_label] = new_label
    return new_labels

def plot_cluster_sizes(labels, title):
    unique, counts = np.unique(labels, return_counts=True)
    plt.figure(figsize=(8, 5))
    plt.bar(unique, counts, alpha=0.7)
    plt.xlabel("Cluster ID")
    plt.ylabel("Size")
    plt.title(title)
    plt.savefig(os.path.join(clusters_folder, f'{title}{with_scanNet_addition}.png'))
    plt.close()

def prevent_imbalanced_clusters(labels, max_ratio=0.3):
    """Reassigns clusters if one cluster is too large relative to others."""
    unique, counts = np.unique(labels, return_counts=True)
    total = sum(counts)
    max_size = total * max_ratio
    if max(counts) > max_size:
        print("Warning: Large cluster detected. Consider alternative clustering or additional constraints.")
    return labels
def louvain_clustering(graph):
    partition = community_louvain.best_partition(graph)
    labels = np.array([partition[i] for i in range(len(partition))])
    number_of_clusters = len(set(labels))
    return number_of_clusters,labels

def leiden_clustering(graph):
    g = ig.Graph.Adjacency((nx.to_numpy_array(graph) > 0).tolist())
    partition = find_partition(g, ModularityVertexPartition)
    labels = np.zeros(graph.number_of_nodes(), dtype=int)
    for i, cluster in enumerate(partition):
        for node in cluster:
            labels[node] = i
    number_of_clusters = len(set(labels))
    return number_of_clusters,labels

def spectral_clustering(graph, n_clusters=80):
    clustering = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', assign_labels='kmeans')
    labels = clustering.fit_predict(graphHomologous)
    return n_clusters,labels

def affinity_propagation(graph):
    graph_dense = graph.toarray()  # Convert sparse matrix to dense numpy array
    clustering = AffinityPropagation(affinity='precomputed')
    labels = clustering.fit_predict(graph_dense)
    number_of_clusters = len(set(labels))
    return number_of_clusters,labels

def connected_components_clustering(graph,directed=False):
    """Perform clustering using connected components."""
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    return n_components, labels

def plot_graph_with_clusters(graph, labels, pos, title="Graph with Clusters", top_n=10):
    """Plot the graph with nodes colored according to their cluster labels, only coloring the top N clusters."""
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_indices = np.argsort(-counts)  # Sort in descending order
    top_labels = unique_labels[sorted_indices[:top_n]]
    
    colors = plt.cm.rainbow(np.linspace(0, 1, top_n))  # Generate a color map for top N clusters

    plt.figure(figsize=(12, 8))
    for label, color in zip(top_labels, colors):
        nx.draw_networkx_nodes(graph, pos, nodelist=[node for node, lbl in enumerate(labels) if lbl == label],
                               node_color=[color], node_size=40, label=f"Cluster {label}")
    nx.draw_networkx_nodes(graph, pos, nodelist=[node for node, lbl in enumerate(labels) if lbl not in top_labels],
                           node_color='grey', node_size=20, label="Other Clusters")  # Smaller grey circles
    nx.draw_networkx_edges(graph, pos, alpha=0.5)
    plt.title(title)
    plt.legend()
    plt.savefig(os.path.join(clusters_folder, f'{title}_graph{with_scanNet_addition}.png'))
    plt.close()

In [6]:
# Convert csr_matrix to NetworkX graph
graph = nx.from_scipy_sparse_array(graphHomologous)
pos = nx.spring_layout(graph)


In [ ]:
_,labels_louvain = louvain_clustering(graph)
labels_louvain = prevent_imbalanced_clusters(labels_louvain)
labels_louvain = reorder_labels_by_cluster_size(labels_louvain)
plot_cluster_sizes(labels_louvain, "LouvainClustering")
plot_graph_with_clusters(graph, labels_louvain,pos, "LouvainClustering")
silhouette_score_louvain = silhouette_score(graphHomologous, labels_louvain)
print(f"Silhouette score for Louvain clustering: {silhouette_score_louvain}")


Silhouette score for Louvain clustering: 0.5212104211116715


In [ ]:
_,labels_leiden = leiden_clustering(graph)
labels_leiden = prevent_imbalanced_clusters(labels_leiden)
labels_leiden = reorder_labels_by_cluster_size(labels_leiden)
plot_cluster_sizes(labels_leiden, "LeidenClustering")
plot_graph_with_clusters(graph, labels_leiden,pos, "LeidenClustering")
silhouette_score_leiden = silhouette_score(graphHomologous, labels_leiden)


In [ ]:
_,labels_spectral = spectral_clustering(graphHomologous)
labels_spectral = prevent_imbalanced_clusters(labels_spectral)
labels_spectral = reorder_labels_by_cluster_size(labels_spectral)
plot_cluster_sizes(labels_spectral, "SpectralClustering")
plot_graph_with_clusters(graph, labels_spectral,pos, "SpectralClustering")
silhouette_score_spectral = silhouette_score(graphHomologous, labels_spectral)
print(f"Silhouette score for Spectral clustering: {silhouette_score_spectral}")

/home/iscb/wolfson/omriyakir/anaconda3/envs/experiment/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Silhouette score for Spectral clustering: 0.4701041439848713


In [ ]:
_,labels_affinity = affinity_propagation(graphHomologous)
labels_affinity = prevent_imbalanced_clusters(labels_affinity)
labels_affinity = reorder_labels_by_cluster_size(labels_affinity)
plot_cluster_sizes(labels_affinity, "AffinityPropagation")
plot_graph_with_clusters(graph, labels_affinity,pos, "AffinityPropagation")
silhouette_score_affinity = silhouette_score(graphHomologous, labels_affinity)
print(f"Silhouette score for Affinity propagation: {silhouette_score_affinity}")

/home/iscb/wolfson/omriyakir/anaconda3/envs/experiment/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Silhouette score for Affinity propagation: 0.26185380811755826


In [ ]:
# make one for connected components
_,labels_connected = connected_components_clustering(graphHomologous)
labels_connected = prevent_imbalanced_clusters(labels_connected)
labels_connected = reorder_labels_by_cluster_size(labels_connected)
plot_cluster_sizes(labels_connected, "ConnectedComponents")
plot_graph_with_clusters(graph, labels_connected,pos, "ConnectedComponents")
silhouette_score_connected = silhouette_score(graphHomologous, labels_connected)
print(f"Silhouette score for Connected components clustering: {silhouette_score_connected}")


Silhouette score for Connected components clustering: 0.2729209628793184


In [13]:
import data_preparation.ScanNet.cath_utils as cath_utils
from data_preparation.ScanNet.db_creation_scanNet_utils import save_as_pickle

# dataset_date = "8_9"
# seq_id = "0.95"
# ASA_THRESHOLD_VALUE = 0.2
# with_scanNet = False
# with_scanNet_addition = '_with_scanNet_' if with_scanNet else ''
# os.makedirs(paths.cath_intermediate_files_path, exist_ok=True)
    
# pssm_folder = os.path.join(paths.PSSM_path,f'PSSM_{dataset_date}', f'seq_id_{seq_id}_asaThreshold_{ASA_THRESHOLD_VALUE}')
# full_pssm_file_path = os.path.join(os.path.join(pssm_folder, f'propagatedPssmWithAsaFile_{seq_id}_asaThreshold_{ASA_THRESHOLD_VALUE}.txt'))
# cath_df = cath_utils.make_cath_df_new(os.path.join(paths.cath_path, "cath_b.20230204.txt"))

# if with_scanNet:
#     scanNet_PSSM = cath_utils.scanNet_PSSM_files_concatenation()
#     ubiq_output_PSSM, scanNet_output_PSSM = cath_utils.propagate_labels_and_concat(full_pssm_file_path,scanNet_PSSM,pssm_folder)
#     full_pssm_file_path += with_scanNet_addition
#     cath_utils.concat_and_remove_duplicates(ubiq_output_PSSM, scanNet_output_PSSM, full_pssm_file_path)

# names_list, sizes_list, sequence_list, full_names_list, pdb_names_with_chains_lists = cath_utils.list_creation(
#     full_pssm_file_path)

    
# structuresDicts = cath_utils.create_dictionaries(names_list, sizes_list, sequence_list, full_names_list,
#                                                      pdb_names_with_chains_lists)
# inCath, notInCath, cnt = cath_utils.count_in_cath(cath_df, structuresDicts)


# cath_utils.find_chains_in_cath(cath_df, structuresDicts)
# cath_utils.add_classifications_for_dict(cath_df, structuresDicts, 4)
    
# matHomologous_path = os.path.join(paths.cath_intermediate_files_path, f'matHomologous_{dataset_date}{with_scanNet_addition}.pkl')
# graphHomologous_path = os.path.join(paths.cath_intermediate_files_path, f'graphHomologous_{dataset_date}{with_scanNet_addition}.pkl')
# if os.path.exists(matHomologous_path) and os.path.exists(graphHomologous_path):
#     matHomologous = cath_utils.load_from_pickle(matHomologous_path)
#     graphHomologous = cath_utils.load_from_pickle(graphHomologous_path)
# else:
#     matHomologous = cath_utils.neighbor_mat_new(structuresDicts)
#     graphHomologous = cath_utils.csr_matrix(matHomologous)
#     save_as_pickle(matHomologous,matHomologous_path)
#     save_as_pickle(graphHomologous,graphHomologous_path)
    
# graph = nx.from_scipy_sparse_array(graphHomologous)


AttributeError: partially initialized module 'yaml' has no attribute 'error' (most likely due to a circular import)

In [1]:
from data_preparation.ScanNet.create_tables_and_weights import create_table,add_model_num_to_dataset
def create_components_for_algorithm(algorithm_name,graph):
    if algorithm_name == "louvain":
        return louvain_clustering(graph)
    elif algorithm_name == "leiden":
        return leiden_clustering(graph)
    elif algorithm_name == "spectral":
        return spectral_clustering(graph)
    elif algorithm_name == "affinity":
        return affinity_propagation(graph)
    elif algorithm_name == "connected":
        return connected_components_clustering(graph)
    else:
        raise ValueError(f"Unknown algorithm name: {algorithm_name}")

test_partition_folder = os.path.join(paths.cath_intermediate_files_path, f'test_partition{with_scanNet_addition}')
os.mkdir(test_partition_folder, exist_ok=True)
algorithm_names = ["louvain", "leiden", "spectral", "affinity", "connected"]
results = {}
for algorithm_name in algorithm_names:
    algorithm_partition_folder = os.path.join(test_partition_folder, algorithm_name)
    os.makedirs(algorithm_partition_folder, exist_ok=True)
    n_clusters, labels = create_components_for_algorithm(algorithm_name,graphHomologous)
    chainDict = cath_utils.components_to_chain_dict(n_clusters, labels,sizes_list,full_names_list)
    cath_utils.divide_pssm(chainDict, full_pssm_file_path,algorithm_partition_folder,with_scanNet_addition)
    for i in range(5):
        pssm_file = os.path.join(algorithm_partition_folder, f"PSSM{with_scanNet_addition}{str(i)}.txt")
        output_file = f'labels_fold{i+1}{with_scanNet_addition}.txt'
        add_model_num_to_dataset(pssm_file, output_file)
    create_table(algorithm_partition_folder ,algorithm_partition_folder,"",with_scanNet_addition)


ModuleNotFoundError: No module named 'data_preparation'

NameError: name 'chainDict' is not defined

In [ ]:
import csv
from collections import defaultdict

def calculate_max_fold_weight(file_path):
    fold_weights = defaultdict(float)
    total_weight = 0.0

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            fold = row['Set']
            weight = float(row['Sample weight'])
            fold_weights[fold] += weight
            total_weight += weight

    max_fold_weight = max(fold_weights.values())
    normalized_max_fold_weight = max_fold_weight / total_weight

    return normalized_max_fold_weight

def get_fold_dict(file_path):
    fold_dict = {}

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            pdb_id = row['PDB ID']
            fold_number = int(row['Set'].split()[-1]) - 1
            fold_dict[pdb_id] = fold_number

    return fold_dict

def get_index_dict(structuresDicts):
    index_dict = {}
    keys = list(structuresDicts.keys())
    for i in range(len(keys)):
        index_dict[keys[i]] = i

    return index_dict

def is_a_cross_edge(fold_dict,index_dict, id1, id2,matHomologous):
    index1 = index_dict[id1]
    index2 = index_dict[id2]
    return matHomologous[index1][index2] == 1 and fold_dict[id1] != fold_dict[id2]


def all_cross_edges(structuresDicts,table):
    index_dict = get_index_dict(structuresDicts)
    fold_dict = get_fold_dict(table)
    print(f' len index_dict: {len(index_dict)}')
    print(f' len fold_dict: {len(fold_dict)}')  
     
    cross_edges = 0
    keys = list(fold_dict.keys())
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            id1 = keys[i]
            id2 = keys[j]
            if is_a_cross_edge(fold_dict, index_dict, id1, id2, matHomologous):
                cross_edges += 1

    return cross_edges


In [ ]:
file_path = '/home/iscb/wolfson/omriyakir/ubinet/datasets/scanNet/data_for_training/8_9_dataset/seq_id_0.9_asaThreshold_0.1/table.csv'
normalized_max_fold_weight = calculate_max_fold_weight(file_path)
print(f'Normalized Max Fold Weight: {normalized_max_fold_weight}')
print(all_cross_edges(structuresDicts,file_path))

Normalized Max Fold Weight: 0.4092827004219409
 len index_dict: 468
 len fold_dict: 462
0
